# Scraping Songfacts.com for releases between 2010-present
Using BeautifulSoup, I am able to pull about 16,000 songs to be sent to the Spotify API for metadata information.

## 1.Obtain URL links for all years

In [1]:
urls=[]
for x in range(1,7):
    urls+=[f'http://www.songfacts.com/released-2018-{x}.php']
for x in range(1,20):
    urls+=[f'http://www.songfacts.com/released-2017-{x}.php']
for x in range(1,21):
    urls+=[f'http://www.songfacts.com/released-2016-{x}.php']   
for x in range(1,27):
    urls+=[f'http://www.songfacts.com/released-2015-{x}.php']
for x in range(1,29):
    urls+=[f'http://www.songfacts.com/released-2014-{x}.php']
for x in range(1,29):
    urls+=[f'http://www.songfacts.com/released-2013-{x}.php']
for x in range(1,27):
    urls+=[f'http://www.songfacts.com/released-2012-{x}.php']
for x in range(1,27):
    urls+=[f'http://www.songfacts.com/released-2011-{x}.php']
for x in range(1,26):
    urls+=[f'http://www.songfacts.com/released-2011-{x}.php']  

In [3]:
from __future__ import print_function, division
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

def get_songfacts_links(urls):
    master=[]
    
    for page in urls:
        response = requests.get(page)
        text = response.text
        soup = BeautifulSoup(text,"html5lib")
        for element in soup.find_all('li')[29:]:
            master+=[str(element)]

    return(master)


In [17]:
z=get_songfacts_links(urls)
        

In [18]:
len(z)

21096

## 2 Minor BeautifulSoup cleaning for a  more Spotify-friendly format

In [19]:
z=[x.replace('<li><a href="/detail.php?id=','') for x in z]
z=[x.replace('" target="_self">','') for x in z]
z=[x.replace('</a>','') for x in z]
z=[x.replace('</li>','') for x in z]

In [22]:
#Removes first 5 digits from the string
z=[x[5:] for x in z]

In [23]:
z=list(set(z))
z.sort()

In [26]:
#When using BeautifulSoup, some random links were picked up that were found on all of the pages. I am going to reduce them to ''.
z=[x.replace('<h2>contribution</h2>\n<div>\n<span>\n<a class="trigger-suggest" href="#">Suggest a Songfact or Artistfact\n<a href="https://forum.songfacts.com">Message Boards\n<a href="http://www.songfacts.com/blog/pages/songfacts_writers/">Songfacts Writers\n</span>\n</div>\n','') for x in z]
z=[x.replace('<h2>follow us</h2>\n<div>\n<span>\n<a class="fb" href="https://www.facebook.com/songfacts" target="_blank">Facebook\n<a class="tw" href="https://twitter.com/songfacts" target="_blank">Twitter\n<a class="yt" href="http://www.youtube.com/user/songfactsdotcom" target="_blank">Youtube\n<a class="em trigger-newsletter" href="#">Newsletter\n</span>\n</div>\n','') for x in z]
z=[x.replace('<h2>information</h2>\n<div>\n<span>\n<a href="http://www.songfacts.com/blog/pages/about_us/">About Us\n<a href="http://www.songfacts.com/blog/pages/terms_of_service/">Terms of Service\n<a href="http://www.songfacts.com/blog/pages/songfacts_privacy_policy_and_legal_notice/">Privacy Policy\n<a href="mailto:feedback@songfacts.com">Contact Us\n<a href="http://www.songfacts.com/blog/pages/songfacts_api/">Songfacts API\n</span>\n</div>\n','') for x in z]
z=[x.replace('<h2>links</h2>\n<div>\n<span>\n<a href="https://calendar.songfacts.com/" target="_blank">Music History Calendar\n<a href="http://www.songplaces.com/" target="_blank">Song Places\n<a href="http://www.songfallacies.com/" target="_blank">Song Fallacies\n</span>\n</div>\n','') for x in z]



In [27]:
z=list(set(z))
z=z[1:]

In [32]:
z=[x.replace("  "," ") for x in z]
z=[x.replace('&amp;','&')for x in z]
songs=pd.DataFrame(z)
songs.to_csv('songs_5_10.csv')

## 3. Appending Spotify data to SongFacts song list

In [14]:
%pylab inline
import spotipy
import csv
import pandas as pd
import numpy as np
import requests
from spotipy import util
import time
unm = "private"
scope = "streaming"

token = util.prompt_for_user_token(
    unm,
    scope,
    client_id='private',
    client_secret='private',
    redirect_uri='http://localhost:8889/tree')
sp = spotipy.Spotify(auth=token)

Populating the interactive namespace from numpy and matplotlib


In [3]:
z=pd.read_csv('songs_5_10.csv',index_col=0)
z=z['0']

In [9]:
uris=[]
for track in z[:2000]:
    try:
        entry= sp.search(q=track,type='track',market='us')['tracks']['items'][0]
        uris += [[entry['uri'],entry['album']['release_date']]]
        print(track+' complete')
    except:
        pass


Hands on You - Ashley Monroe complete
Someone New - Hozier complete
Everybody Lives - Granger Smith complete
I Want To Break Your Mended Heart - Josh Doyle complete
Sweet Summertime - Candlebox complete
On My Heart - School of Seven Bells complete
Blonde - Bridgit Mendler complete
Something I Need - OneRepublic complete
Seigfried - Frank Ocean complete
Mirror - Lil Wayne complete
Shame The Devil - No Malice complete
Til I Get Found - Marc E. Bassy complete
Woman Down - Alanis Morissette complete
Happy Anniversary - Motion City Soundtrack complete
Nobody Wins - Brian Fallon complete
Shut Up And Hold On - Toby Keith complete
End To The Lies - Jane's Addiction complete
All About Tonight - Pixie Lott complete
Love & Sex: Pt 2 - Joe complete
Biggy - Warpaint complete
Pink Matter - Frank Ocean complete
Soon Enough - Aimee Mann complete
The End of the Night - Dropkick Murphys complete
Goodbye Town - Lady Antebellum complete
#1Nite (One Night) - Cobra Starship complete
Trippy - Lil Wayne compl

Let Me Down - Kelly Clarkson complete
Wanted Me Gone - Josh Thompson complete
Not Aretha's Respect (Cops) - Hamell on Trial complete
Road Less Traveled - Sick of It All complete
Cry Baby - Demi Lovato complete
Divine Sorrow - Wyclef Jean complete
White Tiger - Izzy Bizu complete
Soldier's Angel - Stevie Nicks complete
Paul Brown Petty - Brandon Heath complete
I Like Me Better - Lauv complete
Give Me Your Love - Sigala complete
These Four Walls - Little Mix complete
Young - Tulisa complete
I Wanna Love Like That Again - Ronnie Dunn complete
Lights Out, Words Gone - Bombay Bicycle Club complete
Pick Up a Gun - Ray LaMontagne complete
If You Really, Really Love Me - Steel Panther complete
Anywhere For You - John Martin complete
Neon Love - Karmin complete
Something To Believe In - Parachute complete
Summer Forever - Billy Currington complete
Past Life - Tame Impala complete
Wasted Love - Steve Angello complete
Wristband - Paul Simon complete
Fight 'em 'Til You Can't - Anthrax complete
Mil

Father Stretch My Hands Pt. 1 - Kanye West complete
Soul Hacker - Fear Factory complete
Storyline - Hunter Hayes complete
Wasted Years - Maroon 5 complete
Nothing Left But Tears - Buckcherry complete
I Love My Bitches - Rick Ross complete
Still Echoes - Lamb of God complete
Something Good - Estelle complete
Take a Bullet - Geoff Tate complete
Busted - The Dustbowl Revival complete
On My Way - Lea Michele complete
Courtship - Björk complete
Doin' It Right - Daft Punk complete
Ready For Your Love - Gorgon City complete
Over - Blake Shelton complete
Boys in the Wood - Black Lips complete
Everybody Drives Drunk - Love and Theft complete
Ground Control - All Time Low complete
I Like It When You Sleep, for You Are So Beautiful Yet So Unaware of It - The 1975 complete
Keep It Between Us - Kelly Rowland complete
One Drop - Plumb complete
The Joke's On Us - Biffy Clyro complete
Truth In Your Eyes - Kiefer Sutherland complete
Lovin' You Is Fun - Easton Corbin complete
We Bring an Arsenal - Lostp

Whiskey - Jana Kramer complete
Piece By Piece - Anna Calvi complete
Beauty In Tragedy - August Burns Red complete
FU - Miley Cyrus complete
Shameless - Tyler Glenn complete
Kiss Like Your Kiss - Lucinda Williams complete
Thinking About You - Jessie Ware complete
Smile Mama, Smile - Rick Ross complete
Up In Flames - Coldplay complete
Control - Matrix & Futurebound complete
Genghis Khan - Miike Snow complete
White Trash Millionaire - Black Stone Cherry complete
That's Classic - Rizzle Kicks complete
Comeback Kid (That's My Dog) - Brett Dennen complete
Emoji of a Wave - John Mayer complete
I Believe - George Strait complete
Within - Daft Punk complete
Sexy and I Know It - LMFAO complete
Nameless, Faceless - Courtney Barnett complete
Seven Cities Of Gold - Rush complete
Veins - Feeder complete
We Turn Red - Red Hot Chili Peppers complete
Underneath the Tree - Kelly Clarkson complete
Giant In My Heart - Kiesza complete
Pots and Pans - The Kills complete
Zero - Chris Brown complete
Let Me In

Lotus Intro - Christina Aguilera complete
Giving It Up - Mallory Knox complete
Kyoto - Skrillex complete
Go Far - Rudimental complete
Marry Me - Jason Derulo complete
Near Death Experience Experience - Andrew Bird complete
Nightstand - K. Michelle complete
Hype - Calvin Harris complete
Raymond - Brett Eldredge complete
Roses - Krept & Konan complete
Home Again - Elton John complete
Ask Me How I Know - Garth Brooks complete
A Little Bit of Your Heart - Ariana Grande complete
River Song - The Maccabees complete
The Quantum Enigma (Kingdom Of Heaven Part II) - Epica complete
Codes And Keys - Death Cab for Cutie complete
From The Outside - Bizzle complete
Young Forever - Eric Paslay complete
Mexico - Danny Worsnop complete
If I Run - Sara Evans complete
Dance Baby Dance - Chris Cagle complete
The Violent Sleep of Reason - Meshuggah complete
LUV XXX - Aerosmith complete
Wilderness - Cody Simpson complete
No Frauds - Nicki Minaj complete
Overdose - Alessia Cara complete
Stupid Goal - Yael Na

What You Know - Two Door Cinema Club complete
Heartbeat - David Cook complete
Crazy For You - Scars on 45 complete
Hey, Shooter - Rocket Juice And The Moon complete
To Be Given A Body - Torres complete
High School - Nicki Minaj complete
Shake Life - Jeezy complete
Power - Kat Graham complete
Duane Allman - Amy Ray complete
Sing It Loud - k.d. lang complete
Limitedless - Alicia Keys complete
Shadow - Colbie Caillat complete
That Dangerous Age - Paul Weller complete
Satan Pulls the Strings - The Avett Brothers complete
Colors - Halsey complete
Axis - Pet Shop Boys complete
Miracle - Kimbra complete
Tonight - Seether complete
Silence - Marshmello complete
Really Be (Smokin N Drinkin) - YG complete
Jugg - Fetty Wap complete
Acid Rain - Avenged Sevenfold complete
Bonfire - Josh Ritter complete
Painkillers - Brian Fallon complete
Trouble - J. Cole complete
Sex - Mötley Crüe complete
VW Van - Jake Owen complete
We Bleed Metal - Chastain complete
Without End - Anti-Flag complete
Night So Long 

Strip It Down - Luke Bryan complete
Until I'm One With You - Ryan Bingham complete
Sky Walker - Kelly Rowland complete
Break The Rules - Charli XCX complete
Goodbye - Eddie Vedder complete
La Bicicleta - Shakira complete
Mad Love. - JoJo complete
Unbreakable Heart - Three Days Grace complete
Hold Each Other - A Great Big World complete
Here on Earth - Dierks Bentley complete
Four Cypresses - Grizzly Bear complete
Reckless - You Me at Six complete
Second Sound - Druckfarben complete
Crystallize - Lindsey Stirling complete
A--hole - Eminem complete
Mess Is Mine - Vance Joy complete
Meteorite - Mariah Carey complete
Purified - Of Mice & Men complete
I'm a Mess - Ed Sheeran complete
Arlandria - Foo Fighters complete
Payphone - Maroon 5 complete
Intermission: Flower - Zayn complete
Reckless Love - Cory Asbury complete
Dumpster World - Band of Horses complete
The Rambler - Black Stone Cherry complete
Shape Of You - Ed Sheeran complete
Hanging On - Ellie Goulding complete
Power Trip - J. Cole

Mystic Of The East - Van Morrison complete
Photographs - Robert Ellis complete
Country - Angaleena Presley complete
Girl I Met On The Internet - Girli complete
BBD - Azealia Banks complete
Guilty - Paloma Faith complete
A Rumor Of Skin - Stone Sour complete
No Such Thing as a Broken Heart - Old Dominion complete
Rolling Stone - The Weeknd complete
Say You Do - Sigala complete
Numb - Rihanna complete
Genexus - Fear Factory complete
HYFR - Drake complete
Sacrifices - Drake complete
The Mighty Fall - Fall Out Boy complete
Wasted Acres - Grizzly Bear complete
Whispers in the Dark - Mumford & Sons complete
Woman - Kesha complete
Smuckers - Tyler, the Creator complete
Dealbreaker - Rachael Yamagata complete
WW1 Portraits - The Maccabees complete
Man Alive - Funeral for a Friend complete
Greater - MercyMe complete
Because We Can - Bon Jovi complete
Birds - Coldplay complete
Blow - Beyoncé complete
Back To Love - Anthony Hamilton complete
Black Widow - In This Moment complete
First - Cold War 

Yeah Yeah - Willy Moon complete
Wild Season - Banks and Steelz complete
She Hates Me - Big Boi complete
Different Sides - Leonard Cohen complete
Crown - Run The Jewels complete
Dead in the Water - Ellie Goulding complete
Thank You - Busta Rhymes complete
Yemaya - Kele Okereke complete
Inspired - Miley Cyrus complete
One More Sleep - Leona Lewis complete
Options - Luke James complete
Revelation Road - Shelby Lynne complete
Not Another Truck Song - Sugarcane Jane complete
The Shade - Metric complete
Life Jackets - Luke Christopher complete
Runaway - J. Cole complete
Cold as War - Sevendust complete
My Life - 50 Cent complete
Slow, Love, Slow - Nightwish complete
Rising Water - James Vincent McMorrow complete
Like Smoke - Amy Winehouse complete
Piss Off - FFS complete
Disappear - Katy B complete
California - Tim McGraw complete
Easy Love - Sigala complete
Under The Moon - Peace complete
Promise - Ben Howard complete
Rashida - Rufus Wainwright complete
Citycide - The Dead Ships complete
So

In [16]:
for track in z[2000:]:
    try:
        entry= sp.search(q=track,type='track',market='us')['tracks']['items'][0]
        uris += [[entry['uri'],entry['album']['release_date']]]
        print(track+' complete')
    except IndexError:
        print('out of range')

Close To You - Neon Trees complete
Black and White America - Lenny Kravitz complete
Falling Into You - Hillsong Young & Free complete
Girls Talk Boys - 5 Seconds of Summer complete
Tiimmy Turner - Desiigner complete
Roses - The Chainsmokers complete
out of range
Up We Go - Lights complete
out of range
To Love and Die - Jhené Aiko complete
I'll Never Love - Michael Kiwanuka complete
Stoner - Young Thug complete
Legacy - Stryper complete
MP3 - Tesla complete
Ray Charles - Chiddy Bang complete
Gone - My Darkest Days complete
Aftertaste - Shawn Mendes complete
This is the Girl - Patti Smith complete
On My Own - Samantha Barks complete
Slow Cruel Hands Of Time - Band of Horses complete
Know Now - Public Image Ltd. complete
John Wayne - Lady Gaga complete
Lee Van Cleef - Primus complete
The Fighters - LoCash complete
The Fisher King Blues - Frank Turner complete
Trenches - Pop Evil complete
Never Had A Day - Example complete
Runaway - Ed Sheeran complete
Guns and Roses - T.I. complete
Drinki

My Kind of Woman - Justin Moore complete
Afraid of Heights - Billy Talent complete
Higher Than The Sun - Peace complete
Down Down the Deep River - Okkervil River complete
Lavender (Nightfall Remix) - Snoop Dogg complete
Lock The Locks - The Streets complete
Long Hard Road - Alan Jackson complete
Guardian - Alanis Morissette complete
Too High - Blackberry Smoke complete
Get Right - Jimmy Eat World complete
25/8 - Mary J. Blige complete
Revolve - John Newman complete
Still Breathing - Green Day complete
The Last of the Real Ones - Fall Out Boy complete
Local Man Ruins Everything - The Wonder Years complete
Careless World - Tyga complete
It's Your Move - Josh Kelley complete
The Devil Don't Sleep - Brantley Gilbert complete
I Wish You Would - Taylor Swift complete
The Closer I Get To You - Asia complete
Some Brutal Love - Garrett Hazen complete
All Black Everything - Lupe Fiasco complete
El mismo sol (Under the Same Sun) - Alvaro Soler complete
Halo - Cage the Elephant complete
Miracles -

Cry to The World - Renaissance complete
Break Into Your Heart - Iggy Pop complete
Granddaddy's Gun - Blake Shelton complete
...Ready For It? - Taylor Swift complete
Bad As Me - Tom Waits complete
Light It Up - Major Lazer complete
Bang Bang Bang - Selena Gomez & The Scene complete
Our System - OMD complete
Underdogs - Chris Young complete
I Feel Like Ten Men, Nine Dead and One Dying - Band of Skulls complete
I Never Learn - Lykke Li complete
Kitchen Table - Jake Bugg complete
The Saints - Andy Mineo complete
A New Wave - Sleater-Kinney complete
Around My Way (Freedom Ain't Free) - Lupe Fiasco complete
Caution To The Wind - Becky Hill complete
Girls on Boys - Galantis complete
Dawsin's Breek - Ty Dolla $ign complete
Never Never - Korn complete
Southern Comfort Zone - Brad Paisley complete
The Pin - The Goo Goo Dolls complete
Through It All - Device complete
City of Hope - Journey complete
My Indigo - My Indigo complete
Back to the Shack - Weezer complete
The Tally - Mona complete
The Wr

Need You More - Darius Rucker complete
Count It All As Lost - August Burns Red complete
Scars - Boyce Avenue complete
God's Gift - J. Cole complete
Holes In The Sky - M83 complete
Drive - The Shires complete
Déjà Vu - Roger Waters complete
Ice - Kelly Rowland complete
Let The Groove Get In - Justin Timberlake complete
Alone In a Room - Asking Alexandria complete
Cut The Cord - Shinedown complete
Masterpiece - Disclosure complete
Kickin In - Adam Lambert complete
Leaving the Monsters Behind - The Jayhawks complete
Buzzkill - Luke Bryan complete
Little Red Wagon - Miranda Lambert complete
Sober - G-Eazy complete
This Is Your Sword - Bruce Springsteen complete
out of range
Are You With Me - Easton Corbin complete
Drunk Americans - Toby Keith complete
Cry of Achilles - Alter Bridge complete
Hi Ho the Righteous - Jonathan Wilson complete
Gasoline - Demon Hunter complete
Last First Kiss - One Direction complete
Loved By a Workin' Man - Pistol Annies complete
Candidate - Johnny Marr complete


T-Shirt - Migos complete
Bradley Manning - Cass McCombs complete
Donald Trump - Mac Miller complete
We Found Love - Rihanna complete
Angels In Chelsea - Rachel Platten complete
out of range
Simple as This - Jake Bugg complete
Sleep - The Roots complete
Sativa - Jhené Aiko complete
Y.U. Mad - Birdman complete
Meth Labs & Moonshine - Mini Thin complete
Torus - Sub Focus complete
Notorious - The Saturdays complete
Vice - Lisbon complete
out of range
Recess - Skrillex complete
A Life Embossed - Protest the Hero complete
Feelin' It - Scotty McCreery complete
Picasso Baby - Jay-Z complete
This I Know - Demon Hunter complete
Hostage - Klangstof complete
Daughters - Wild Beasts complete
Butterfly - Tracy Lawrence complete
out of range
Kisses Down Low - Kelly Rowland complete
Tomorrow's Food - Incubus complete
Trap Queen - Fetty Wap complete
Sunlight - The Magician complete
Bitches and Bottles (Let's Get It Started) - DJ Khaled complete
Help - The Ting Tings complete
Preacher's Daughter - Maggi

...

Supersymmetry - Arcade Fire complete
out of range
Something Wicked This Way Comes - Lucinda Williams complete
Stay Stay Stay - Taylor Swift complete
The Evolution Of Man - Example complete
Wasted - Tiesto complete
Black Opium - Rick Ross complete
Hotel Ceiling - Rixton complete
A Thousand Million Reasons - Colin Hay complete
Something's Gotta Give - All Time Low complete
No Friend - Paramore complete
Déjà Vu - Twenty88 complete
Sunshine Riptide - Fall Out Boy complete
Not a Bad Man - Patty Griffin complete
Ghost River - Nightwish complete
out of range
Clouds - One Direction complete
In Memory Of… - Silverstein complete
Runaway - The Kooks complete
So Now What - The Shins complete
Ayla - The Maccabees complete
The Other Side - Alter Bridge complete
Goodbye Back - Justin Moore complete
In the Garden of Edie - Paul Simon complete
Celebrate - Mika complete
Guitar Man - Kip Moore complete
Amar Pelos Dois - Salvador Sobral complete
Blue Ocean Floor - Justin Timberlake complete
Get 'Em Up - N

So Am I - Ty Dolla $ign complete
The Beautiful & Damned - G-Eazy complete
WHITECAPS - Prince complete
Frankenstein - Lenny Kravitz complete
Slave - Yeah Yeah Yeahs complete
Long Distance Lullaby - Martina McBride complete
Cast The First Stone - Slayer complete
The Circle - Black Country Communion complete
All I Ever Wanted - Vance Joy complete
Everything Is Awesome - Tegan and Sara complete
Bathed in Grey - King Krule complete
Family Don't Matter - Young Thug complete
They Don't Know - Jason Aldean complete
Woman to Woman - Keyshia Cole complete
Mojave Phone Booth - Steve Barton complete
Too Many Miles - Nils Lofgren complete
Get on Your Knees - Nicki Minaj complete
Through The Dark - One Direction complete
Prehistoric Dog - Red Fang complete
House - Patrick Wolf complete
Carry On - Fun. complete
One Sunday Morning (Song for Jane Smiley's Boyfriend) - Wilco complete
Phone Down - Erykah Badu complete
Nothing Stands in Our Way - Lacuna Coil complete
Together - Ella Eyre complete
Navajo R

Ghost On The Dance Floor - blink-182 complete
Just a Little Boy (for Chester Burnett) - Swans complete
Fatal Distraction - Mario complete
The Sparrow - Mastodon complete
Face the Sun - Miguel complete
Moves Like Jagger - Maroon 5 complete
Desperado - Azealia Banks complete
Like A Rose - Ashley Monroe complete
Sun Goes Down - David Guetta complete
Wave - Beck complete
Pyramid - Two Door Cinema Club complete
Uh Huh - Julia Michaels complete
Something To Believe In - Sister Hazel complete
Absolution Calling - Incubus complete
The Prophet - Jamie T. complete
Bored to Death - Eric Hutchinson complete
Doing it Our Way - Gloriana complete
Circles Around the Sun - Dispatch complete
If You Leave Me Now - Foxes complete
Offering - Chelsea Wolfe complete
Touch - Shift K3Y complete
Christmas In The Sand - Colbie Caillat complete
Forget To Remember - Megadeth complete
Let Legend Mark Me as the King - Christopher Lee complete
Hold On - Idris Elba complete
Take a Chance - Monica complete
Lights Out -

We Run the Night - Havana Brown complete
Take Me As I Am - David Cook complete
out of range
Water Under the Bridge - Adele complete
Funk 50 - Joe Walsh complete
So I Could Find My Way - Enya complete
Lions - Skillet complete
Study In Blue - Paul Weller complete
Make Me Better - James Blunt complete
My House by the Water - Mac DeMarco complete
Patty Cake - Kodak Black complete
The Original High - Adam Lambert complete
Salvatore - Lana Del Rey complete
Sad Girl - Lana Del Rey complete
Asking for a Friend - Devin Dawson complete
Night & Day - Baha Men complete
All We Have Is Now - The Modern Electric complete
Nuh Ready Nuh Ready - Calvin Harris complete
Angels - Chance the Rapper complete
Sidewalks - The Weeknd complete
The Good Good - Snoop Lion complete
Dare You - Hardwell complete
Blood, Sweat, Tears - Future complete
We're All Friends - DEC3 complete
Stars - Sixx: A.M. complete
100 Letters - Halsey complete
King City - Swim Deep complete
out of range
Sounds Like Balloons - Biffy Clyro

All Night - Chance the Rapper complete
Gone - Afrojack complete
The Lie - Of Mice & Men complete
A Different World - Korn complete
The Long Way Home - Rosanne Cash complete
When Christmas Comes Around - Matt Terry complete
Speed Roadster - David Lynch complete
Neat Little Rows - Elbow complete
Pop That Lock - Adam Lambert complete
Cruisin' - Colbie Caillat complete
Le Pingouin - Carla Bruni complete
Better Man - Little Big Town complete
Back to Earth - Steve Aoki complete
out of range
Nothing Arrived - Villagers complete
Plane Song - Katie Melua complete
Take You - Justin Bieber complete
Alone Again - Betty Who complete
Four Years of Chances - Margo Price complete
Out Of The Dark - Klaxons complete
Low Times - School of Seven Bells complete
Snake Oil - Foals complete
Stay - Zedd complete
Dirty Rotten Bastards - Green Day complete
Centuries - Fall Out Boy complete
Punkyoungirl - Adam Ant complete
Wet Dreamz - J. Cole complete
Once For All - Paul Baloche complete
How Big, How Blue, How B

In [17]:
len(uris)

20773

In [20]:
uris

[['spotify:track:6SEDE7wvAnAtfdhu1zbUzi', '2018-04-20'],
 ['spotify:track:0efT4YKQLQx2YHbp6vgRX8', '2014-10-07'],
 ['spotify:track:2Bk0eoq8bN02TbqMlCLOHz', '2017-10-27'],
 ['spotify:track:1Lov8H1IsSBRdv5xWJoAS6', '2012-10-22'],
 ['spotify:track:38KkaiMlZHZ72cuoCgltVX', '2012-04-03'],
 ['spotify:track:02kGA6T5LwlhMm253ZlFMY', '2016-02-12'],
 ['spotify:track:1kGMxiOni0S03K9hlFpdCe', '2012-01-01'],
 ['spotify:track:4KANJH1baadr3U7XsVbM17', '2014-01-01'],
 ['spotify:track:1BViPjTT585XAhkUUrkts0', '2016-08-20'],
 ['spotify:track:05pdoheuKPSotkjMgIVX6I', '2011-01-01'],
 ['spotify:track:6OOiErBsbuMQIoEZTzEkNz', '2013-08-18'],
 ['spotify:track:3UReTN3heQsznBG8edEcTU', '2017-10-13'],
 ['spotify:track:2MlOpig13durvtL9JdTRn5', '2013-01-01'],
 ['spotify:track:7rMDS4aYSNIKsEgB9IXhxh', '2012-06-12'],
 ['spotify:track:1EQ9DvgyNEXErxzWIQsGRG', '2016-03-11'],
 ['spotify:track:2mYs6cavjhAnt3l2YiMAks', '2013-01-01'],
 ['spotify:track:7zQEPFd7ubM7PaNRx2UMBG', '2011-10-14'],
 ['spotify:track:1Fx9aiORwv0cx5

In [21]:
cleaned=pd.DataFrame(uris)
tracks=cleaned[0]

In [23]:
len(set(tracks))

16351

## 4. Getting data based off of URIs

In [25]:
chunks = [tracks[x:x+50] for x in range(0, len(tracks), 50)]
song_details=[]
for x in chunks:
    song_details+=sp.tracks(x)['tracks']

In [26]:
song_df=pd.DataFrame(song_details)

In [30]:
song_df.drop(columns=['available_markets','disc_number','external_urls',\
                      'external_ids','href','is_local','preview_url','track_number','type'],inplace=True)

In [32]:
album1=song_df['album'].apply(pd.Series)
album1=album1['release_date']

In [34]:
song_df['collabs']=[len(x)-1 for x in song_df['artists']]
song_df['artist_name']=[x[0].get('name') for x in song_df['artists']]
song_df['artist_uri']=[x[0].get('uri') for x in song_df['artists']]
song_info=pd.concat([song_df, album1], axis=1, join_axes=[song_df.index])



### Pulling audio features in a separate call


In [27]:
features=[]
for x in chunks:
    features+= sp.audio_features(x)

In [28]:
y=list(range(len(features)))

feat=list(zip(y,features))



In [29]:
feat=pd.DataFrame(feat)
audio=feat[1].apply(pd.Series)


### Bringing it together

In [35]:
spotify_info=pd.concat([song_info, audio], axis=1, join_axes=[song_info.index])

In [36]:
spotify_info.to_csv('spotify5_10.csv')

## 5. We may be missing some artists that weren't covered in the orginal billboard pull. Let's cover all the artists we're pulling.

In [37]:
artistry=list(spotify_info.artist_uri.unique())

In [38]:
artist_information=[]

for x in artistry[:]:
    artist_info=sp.artist(x)
    row=({'uri': x,
          'artist':artist_info['name'],
          'genres':artist_info['genres'],
          'followers':artist_info['followers'].get('total')})
    artist_information.append(row)
    print(artist_info['name']+' appended')

Ashley Monroe appended
Hozier appended
Granger Smith appended
Josh Doyle appended
Candlebox appended
School Of Seven Bells appended
Bridgit Mendler appended
OneRepublic appended
Frank Ocean appended
Lil Wayne appended
No Malice appended
Marc E. Bassy appended
Alanis Morissette appended
Motion City Soundtrack appended
Brian Fallon appended
Toby Keith appended
Jane's Addiction appended
Pixie Lott appended
Joe appended
Warpaint appended
Aimee Mann appended
Dropkick Murphys appended
Lady Antebellum appended
Cobra Starship appended
Juicy J appended
Andy Grammer appended
Meat Loaf appended
Paul McCartney appended
One Direction appended
Sigma appended
Kehlani appended
Prophets Of Rage appended
Asking Alexandria appended
Lana Del Rey appended
Elle King appended
Drake appended
Jack Ü appended
Nightwish appended
Young the Giant appended
Mark Knopfler appended
Greyson Chance appended
Cage The Elephant appended
Julian Lennon appended
Cover Pop appended
Marianne Faithfull appended
Nothing More appe

Greg Bates appended
Flying Lotus appended
Fear Factory appended
Hunter Hayes appended
Maroon 5 appended
Buckcherry appended
Geoff Tate appended
The Dustbowl Revival appended
Lea Michele appended
Björk appended
Daft Punk appended
Gorgon City appended
Blake Shelton appended
Black Lips appended
Love and Theft appended
All Time Low appended
The 1975 appended
Kelly Rowland appended
Plumb appended
Kiefer Sutherland appended
Easton Corbin appended
lostprophets appended
Usher appended
Soulfly appended
Howler appended
Bebe Rexha appended
Wale appended
Daughtry appended
Vampire Weekend appended
Pentatonix appended
The Chemical Brothers appended
Father John Misty appended
Una Healy appended
Pusha T appended
Jake Bugg appended
Waka Flocka Flame appended
Amy Winehouse appended
Act of Defiance appended
Anberlin appended
Mac Miller appended
Limp Bizkit appended
A$AP Rocky appended
Jeremih appended
Purity Ring appended
Megadeth appended
The Antlers appended
Prince appended
Kendrick Lamar appended
John

Chairlift appended
KYLE appended
Crown The Empire appended
Volbeat appended
Earl Sweatshirt appended
Ben Folds Five appended
Toni Braxton appended
Naughty Boy appended
Flo Rida appended
Don Henley appended
Robin Thicke appended
Vertical Horizon appended
Sparks appended
Sinéad O'Connor appended
Lori McKenna appended
Boston appended
AURORA appended
Trace Adkins appended
Chuck Wicks appended
Francesca Battistelli appended
Julia Holter appended
Meghan Linsey appended
Tegan and Sara appended
Maverick Sabre appended
LCD Soundsystem appended
CL appended
Tim McGraw appended
Hilary Duff appended
Amy LaVere appended
Richie Kotzen appended
Instrumental King appended
Tove Lo appended
Christina Perri appended
Caribou appended
retrying ...1secs
retrying ...1secs
Krazed Platinum appended
Lianne La Havas appended
Control appended
Thousand Foot Krutch appended
Ólafur Arnalds appended
Rich Homie Quan appended
Papa Roach appended
David Cook appended
Scars On 45 appended
Rocket Juice & The Moon appended
T

Hilltop Hoods appended
Big Time Rush appended
Chance The Rapper appended
Tech N9ne Collabos appended
Jason Isbell and the 400 Unit appended
McFly appended
Mushroomhead appended
Kasabian appended
DOWN appended
Jackie Evancho appended
New Party Crew appended
Margo Price appended
Sweet Little Band appended
Alesana appended
The Darkness appended
Kate Pierson appended
The Cult appended
Porcelain Black appended
SBTRKT appended
The Neighbourhood appended
Cass McCombs appended
Hawksley Workman appended
Flatline appended
Lenny Kravitz appended
Hillsong Young & Free appended
Lights appended
Michael Kiwanuka appended
Young Thug appended
Chiddy Bang appended
My Darkest Days appended
Shawn Mendes appended
Samantha Barks appended
Primus appended
Pop Evil appended
Bucky Covington appended
Lloyd Banks appended
Emerson, Lake & Palmer appended
Glass Animals appended
Dawin appended
The Johnstones appended
Billy Ray Cyrus appended
The-Dream appended
Brody Dalle appended
Montell Jordan & Victory World Musi

The Lacs appended
Joan Armatrading appended
Miguel appended
Chart Topping Karaoke appended
Travis Scott appended
Years & Years appended
Canaan Smith appended
James Taylor appended
Astro Tunes appended
Grace VanderWaal appended
The Maine appended
Dave Davies appended
Dan Mangan appended
Not3s appended
Harper Simon appended
Ziggy Marley appended
Roger Sanchez appended
Shane Filan appended
Benjamin Gibbard appended
Karaoke - Ameritz appended
Deaf Havana appended
Chlöe Howl appended
Kid Cudi appended
The Janoskians appended
Lisa Hannigan appended
Joss Stone appended
Drowning Pool appended
Barry Gibb appended
Underworld appended
Primal Scream appended
Sway appended
Pearl Jam appended
Steven Curtis Chapman appended
Gillian Welch appended
Jessica Sanchez appended
Flume appended
R. City appended
Jimi Goodwin appended
White Denim appended
Chris August appended
The Temperance Movement appended
Meridian Dan appended
Victoria Justice appended
Eli Young Band appended
Randy Houser appended
Little Dr

Siedah Garrett appended
Cheryl appended
The Kooks appended
Music Maverick appended
Deorro appended
Los Lonely Boys appended
Chip appended
Van Halen appended
Burn Halo appended
JP Cooper appended
Grace appended
Jason Gray appended
Electro Velvet appended
Hoobastank appended
Zendaya appended
The Common Linnets appended
Yo Gotti appended
Marika Hackman appended
David Correy appended
The War On Drugs appended
Country Joe McDonald appended
Benmont Tench appended
Ra Ra Riot appended
Guy Garvey appended
Fantasia appended
Karaoke Galaxy appended
Rev Theory appended
Simply Red appended
Sheppard appended
Michael Cera appended
Buck 22 appended
Joey Hyde appended
Trent Harmon appended
Lucy Rose appended
Stooshe appended
Jacquees appended
Denman appended
Ashley McBryde appended
Courteeners appended
Caleb Johnson appended
Conor Maynard appended
YoungBoy Never Broke Again appended
Bow Wow appended
We Are Messengers appended
The Flaming Lips appended
Steven Tyler appended
Jazmine Sullivan appended
Lee

The Hold Steady appended
Dia Frampton appended
Joey + Rory appended
Alabama appended
Josh Gracin appended
Rusted Root appended
Shovels & Rope appended
Yuna appended
Clare Dunn appended
Craig Wedren appended
Set It Off appended
Destiny's Child appended
Alexandra Burke appended
Chris Rene appended
Jamala appended
Ashley Campbell appended
Stephen Malkmus & The Jicks appended
J Balvin appended
Matthew Koma appended
The Downtown Fiction appended
Lacuna Coil appended
Loverboy appended
Charred Walls Of The Damned appended
Lil Snupe appended
Havana Brown appended
Phosphorescent appended
Pope Francis appended
Alvvays appended
Sons of the Sea appended
Jagged Edge appended
Lieutenant appended
Baha Men appended
Susanne Sundfør appended
Mr. Criminal appended
Yasmin appended
Rex Orange County appended
Joan Baez appended
Formation appended
Not For Long appended
Rebelution appended
Anvil appended
ALMA appended
Thursday appended
La Toya Jackson appended
Fiona Apple appended
Laura Welsh appended
retryin

Outasight appended
Chyno appended
Jukebox Heaven appended
Squeeze appended
Lilly Wood and The Prick appended
I Break Horses appended
The Fall appended
Amanda Palmer appended
Glenn Molloy appended
De La Soul appended
The Districts appended
YFN Lucci appended
Restless Heart appended
Ray Barbee appended
MC YOGI appended
Ryn Weaver appended
Alice Glass appended
Emery appended
The Amsterdam Red Light District appended
Travie McCoy appended
Third Eye Blind appended
John Waite appended
2015 Fuelled Pop appended
Pia Mia appended
Rico Love appended
KOLARS appended
Idina Menzel appended
Jordan Knight appended
Paul Mealor appended
Deacon Blue appended
TQX appended
Alex Day appended
Iced Earth appended
Frank + Derol appended
Edward Sharpe & The Magnetic Zeros appended
Diane Warren appended
Gang Of Four appended
Grace Chapman appended
Nick Fradiani appended
Kathleen Edwards appended
Faith Hill appended
India.Arie appended
Megan Nicole appended
Procol Harum appended
Soilwork appended
Andrew Rannells

Mick Jagger appended
Blood Red Shoes appended
Chris Willis appended
Patrick Wolf appended
retrying ...1secs
retrying ...1secs
Justine Skye appended
Audien appended
Ani DiFranco appended
Anne Hathaway appended
The Hives appended
Mark Collie appended
Ryan Stevenson appended
Paul Buchanan appended
The Corrs appended
Calhoun appended
Coal Chamber appended
RZA appended
Otto Knows appended
Quiet Riot appended
Sara Ramirez appended
Impending Doom appended
Ronnie Spector appended
Conchita Wurst appended
Chart Nation appended
Hillsong Worship appended
Audio Adrenaline appended
BlocBoy JB appended
Cedric Gervais appended
Fischerspooner appended
Frank Iero And The Patience appended
Al Atkins appended
Clay Walker appended
Ameritz Digital Karaoke appended
Morgan Wallen appended
Straight No Chaser appended
Natalie Hemby appended
Natasha Bedingfield appended
Platizone appended
Kamelot appended
Dan Croll appended
Pretty Little Horses appended
DREAMCAR appended
Motion Music appended
Beware Of Darkness 

In [39]:
artist_info=pd.DataFrame(artist_information)
artist_info.genres=artist_info.genres.astype(str)
artist_info=artist_info[artist_info.genres!='[]']

In [40]:
artist_info.to_csv('artists_5_10_1.csv')